In [1]:
import cv2
import os
import csv
import numpy as np
import matplotlib.pyplot as plt

In [7]:
dataset_idx_st=0
dataset_idx_end = 90
box_dim = [-32, 32, -32, 32]
np.random.seed(63986)
plotting=False

In [8]:
# algorithm: https://www.hackerearth.com/practice/notes/how-to-check-if-two-rectangles-intersect-or-not/
def check_overlap(box1, box2):
    if (box1[0] > box2[1]) or (box2[0] > box1[1]) or (box1[2] > box2[3]) or (box2[2] > box1[3]):
        return False
    else:
        return True

In [9]:
X = np.array([])
y = []
# For each image
for i in range(dataset_idx_st,dataset_idx_end+1):
    image_string = '{:08d}'.format(i)
    image_file = "../Data/Vehicules1024/"+image_string+"_ir.png"
    annotation_file = "../Data/Annotations1024/"+image_string+".txt"
    # Make sure both the image file and its annotations exist
    if os.path.isfile(image_file) and os.path.isfile(annotation_file):
        print("Processing ", image_string)
        # Read the annotations file and eliminate any vehicles too close to the edge
        selected_v = np.array([])
        with open(annotation_file) as f:
            readcsv = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
            for r in readcsv:
                corners = np.round(np.column_stack((r[0], r[0], r[1], r[1])) + box_dim)
                if corners.min() >= 0 and corners.max() <= 1023:
                    print("Object in bounds ", corners)
                    if selected_v.shape[0] == 0:
                        print("First object")
                        selected_v = corners
                    else:
                        selected_v = np.append(selected_v, corners, axis=0)
                else:
                    print("Object out of bounds - ", corners)
        print("Found ", selected_v.shape[0], " in-bounds vehicles")
        
        # for each in-bounds vehicle, grab a background image
        bg_chip = np.array([])
        for v in range(0, selected_v.shape[0]):
            clean_bg = False
            while (not clean_bg):
                corner = np.random.uniform(65, 900, 2)
                corners = np.round(np.column_stack((corner[0], corner[0], corner[1], corner[1]))+box_dim)
                is_overlap = False
                for vv in range(0, selected_v.shape[0]):
                    if check_overlap(selected_v[vv,:], corners[0,:]):
                        is_overlap = True
                if ( not is_overlap):
                    clean_bg = True
            if bg_chip.shape[0] == 0:
                bg_chip = corners
            else:
                bg_chip = np.append(bg_chip, corners, axis=0)
        print("Created ", bg_chip.shape[0], " background patches")
            
        # Eliminate any overlapping vehicles
        overlap_idx = []
        if selected_v.shape[0]>1:
            for v in range(0,selected_v.shape[0]):
                other_v = np.delete(selected_v, v, axis=0) #remove this one
                v_overlap = False
                for vo in range(0, other_v.shape[0]):
                    if check_overlap(selected_v[v,:], other_v[vo,:]):
                        v_overlap = True
                if v_overlap:
                    overlap_idx.append(v)
        selected_v_post_overlap = np.delete(selected_v, overlap_idx, axis=0)
        print("Found ", selected_v_post_overlap.shape[0], " non-overlapping vehicles")
        
        # Read the image
        img = cv2.imread(image_file)
        img = img[:,:,0]; #only a single channel for IR
        if plotting:
            plt.imshow(img, cmap='gray');
            plt.title(image_string)
            plt.show()
        
        # Display each of the selected vehicles
        if selected_v_post_overlap.shape[0] > 0:
            for v in range(0,selected_v_post_overlap.shape[0]):
                these_corners = selected_v_post_overlap[v,:].astype(int)
                patch = img[these_corners[2]:these_corners[3], these_corners[0]:these_corners[1]];
                
                if X.shape[0] == 0:
                    X = np.zeros((64,64,1))
                    X[:,:,0] = patch
                else:
                    X = np.dstack((X, patch))
                y.append(1)
                
                
                these_corners = bg_chip[v,:].astype(int)
                bg_patch = img[these_corners[2]:these_corners[3], these_corners[0]:these_corners[1]];
                
                X = np.dstack((X, bg_patch))
                y.append(0)
                
                if plotting:
                    plt.subplot(2,selected_v_post_overlap.shape[0],v+1)
                    plt.imshow(patch, cmap='gray');
                    plt.axis('off')
                    plt.subplot(2,selected_v_post_overlap.shape[0],selected_v_post_overlap.shape[0]+v+1)
                    plt.imshow(patch, cmap='gray');
                    plt.axis('off')
            
            if plotting:
                plt.show()
        else:
            print("No selected vehicles")
        print("\n")
    else:
        print("Image ", image_file, " and/or annotation ", annotation_file, "does not exist")

Processing  00000000
Object out of bounds -  [[ 549.  613.  977. 1041.]]
Found  0  in-bounds vehicles
Created  0  background patches
Found  0  non-overlapping vehicles
No selected vehicles


Processing  00000001
Object in bounds  [[313. 377. 780. 844.]]
First object
Object in bounds  [[381. 445. 779. 843.]]
Found  2  in-bounds vehicles
Created  2  background patches
Found  2  non-overlapping vehicles


Processing  00000002
Object in bounds  [[575. 639. 814. 878.]]
First object
Found  1  in-bounds vehicles
Created  1  background patches
Found  1  non-overlapping vehicles


Processing  00000003
Object in bounds  [[845. 909. 859. 923.]]
First object
Found  1  in-bounds vehicles
Created  1  background patches
Found  1  non-overlapping vehicles


Processing  00000004
Object in bounds  [[260. 324. 592. 656.]]
First object
Object in bounds  [[303. 367. 439. 503.]]
Found  2  in-bounds vehicles
Created  2  background patches
Found  2  non-overlapping vehicles


Image  vedai/Vehicules1024/000000



Processing  00000033
Object in bounds  [[197. 261. 730. 794.]]
First object
Object in bounds  [[464. 528. 489. 553.]]
Object in bounds  [[457. 521. 521. 585.]]
Object in bounds  [[571. 635. 867. 931.]]
Object in bounds  [[594. 658. 862. 926.]]
Object out of bounds -  [[ 966. 1030.  875.  939.]]
Object out of bounds -  [[ 992. 1056.  889.  953.]]
Found  5  in-bounds vehicles
Created  5  background patches
Found  1  non-overlapping vehicles


Image  vedai/Vehicules1024/00000034_ir.png  and/or annotation  vedai/Annotations1024/00000034.txt does not exist
Processing  00000035
Object in bounds  [[ 20.  84. 618. 682.]]
First object
Object in bounds  [[ 32.  96. 641. 705.]]
Object in bounds  [[478. 542. 302. 366.]]
Object in bounds  [[666. 730. 592. 656.]]
Found  4  in-bounds vehicles
Created  4  background patches
Found  2  non-overlapping vehicles


Processing  00000036
Object in bounds  [[359. 423. 174. 238.]]
First object
Object in bounds  [[428. 492. 784. 848.]]
Found  2  in-bounds veh



Processing  00000053
Object in bounds  [[574. 638.  22.  86.]]
First object
Object in bounds  [[423. 487.  83. 147.]]
Object in bounds  [[164. 228.  19.  83.]]
Object in bounds  [[722. 786. 568. 632.]]
Object in bounds  [[896. 960. 566. 630.]]
Object in bounds  [[902. 966. 622. 686.]]
Object in bounds  [[138. 202. 830. 894.]]
Found  7  in-bounds vehicles
Created  7  background patches
Found  5  non-overlapping vehicles


Processing  00000054
Object in bounds  [[ 72. 136. 335. 399.]]
First object
Object in bounds  [[ 86. 150. 444. 508.]]
Object in bounds  [[102. 166. 554. 618.]]
Object in bounds  [[115. 179. 661. 725.]]
Found  4  in-bounds vehicles
Created  4  background patches
Found  4  non-overlapping vehicles


Processing  00000055
Object in bounds  [[488. 552. 154. 218.]]
First object
Object in bounds  [[471. 535. 173. 237.]]
Object in bounds  [[ 51. 115. 480. 544.]]
Object in bounds  [[715. 779. 229. 293.]]
Found  4  in-bounds vehicles
Created  4  background patches
Found  2  no



Processing  00000080
Object in bounds  [[ 87. 151. 324. 388.]]
First object
Object in bounds  [[150. 214. 412. 476.]]
Found  2  in-bounds vehicles
Created  2  background patches
Found  2  non-overlapping vehicles


Processing  00000081
Object in bounds  [[ 22.  86. 121. 185.]]
First object
Object in bounds  [[  8.  72. 288. 352.]]
Object in bounds  [[799. 863.  26.  90.]]
Object in bounds  [[825. 889.  28.  92.]]
Object in bounds  [[891. 955.  56. 120.]]
Object in bounds  [[629. 693. 186. 250.]]
Object in bounds  [[317. 381. 517. 581.]]
Object in bounds  [[241. 305. 659. 723.]]
Object in bounds  [[ 55. 119. 931. 995.]]
Object out of bounds -  [[ 129.  193.  991. 1055.]]
Found  9  in-bounds vehicles
Created  9  background patches
Found  7  non-overlapping vehicles


Processing  00000082
Object in bounds  [[616. 680.  94. 158.]]
First object
Object in bounds  [[337. 401. 509. 573.]]
Object in bounds  [[218. 282. 515. 579.]]
Found  3  in-bounds vehicles
Created  3  background patches
Fo

In [10]:
#Format the resulting data and write it to a file
print("X shape=",X.shape)
X = np.transpose(X,(2, 0, 1))
y = np.asarray(y)
print("X shape=",X.shape," y shape=",y.shape)

X shape= (64, 64, 472)
X shape= (472, 64, 64)  y shape= (472,)


In [11]:
from sklearn import svm, metrics, model_selection

# Simple model training with a single training/testing data split
X_flat = X.reshape((len(X), -1))
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_flat, y, test_size=0.25, random_state=23)
clf = svm.SVC(kernel='rbf', gamma=0.001)
clf.fit(X_train,y_train)

#Check performance
y_pred = clf.predict(X_test)

print("Accuracy score:\n", metrics.accuracy_score(y_test,y_pred))

Accuracy score:
 0.4745762711864407


In [13]:
# Same thing, with a grid-searching cross validation
#parameters = [
#    {'kernel':['rbf'], 'gamma':np.logspace(-6, -3, 20)}
#]
parameters = [
    {'kernel':['rbf'], 'gamma':np.logspace(-6, -3, 20)},
    {'kernel':['linear'], 'C':np.logspace(-5, 4, 20)}
]
clf=svm.SVC()
cv_clf=model_selection.GridSearchCV(estimator=clf, param_grid=parameters, cv=5, n_jobs=-1);
cv_clf.fit(X_train,y_train)
print("Best estimator:\n", cv_clf.best_estimator_)

#Check performance
y_pred = cv_clf.predict(X_test)

print("Accuracy score:\n", metrics.accuracy_score(y_test,y_pred))

Best estimator:
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-06, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Accuracy score:
 0.8305084745762712
